## Image Cleaner Widget

fastai offers several widgets to support the workflow of a deep learning practitioner. The purpose of the widgets are to help you organize, clean, and prepare your data for your model. Widgets are separated by data type.

In [ ]:
from fastai.vision import *
from fastai.widgets import DatasetFormatter, ImageCleaner, ImageDownloader, download_google_images
from fastai.gen_doc.nbdoc import *

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,error_rate,time
0,0.184135,0.103444,0.037291,00:06
1,0.110252,0.082977,0.028459,00:05


In [ ]:
learn.save('stage-1')

We create a databunch with all the data in the training set and no validation set (DatasetFormatter uses only the training set)

In [ ]:
db = (ImageList.from_folder(path)
                   .split_none()
                   .label_from_folder()
                   .databunch())

In [ ]:
learn = cnn_learner(db, models.resnet18, metrics=[accuracy])
learn.load('stage-1');

In [ ]:
show_doc(DatasetFormatter)

<h2 id="DatasetFormatter" class="doc_header"><code>class</code> <code>DatasetFormatter</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L14" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>DatasetFormatter</code>()

<div class="collapse" id="DatasetFormatter-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>DatasetFormatter</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Returns a dataset with the appropriate format and file indices to be displayed.  

The [`DatasetFormatter`](/widgets.image_cleaner.html#DatasetFormatter) class prepares your image dataset for widgets by returning a formatted [`DatasetTfm`](/vision.data.html#DatasetTfm) based on the [`DatasetType`](/basic_data.html#DatasetType) specified. Use `from_toplosses` to grab the most problematic images directly from your learner. Optionally, you can restrict the formatted dataset returned to `n_imgs`.

In [ ]:
show_doc(DatasetFormatter.from_similars)

<h4 id="DatasetFormatter.from_similars" class="doc_header"><code>from_similars</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L35" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-from_similars-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>from_similars</code>(**`learn`**, **`layer_ls`**:`list`=***`[0, 7, 2]`***, **\*\*`kwargs`**)

<div class="collapse" id="DatasetFormatter-from_similars-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-from_similars-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>from_similars</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Gets the indices for the most similar images.  

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.widgets.image_cleaner import * 

In [ ]:
show_doc(DatasetFormatter.from_toplosses)

<h4 id="DatasetFormatter.from_toplosses" class="doc_header"><code>from_toplosses</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L16" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-from_toplosses-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>from_toplosses</code>(**`learn`**, **`n_imgs`**=***`None`***, **\*\*`kwargs`**)

<div class="collapse" id="DatasetFormatter-from_toplosses-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-from_toplosses-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>from_toplosses</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Gets indices with top losses.  

In [ ]:
show_doc(ImageCleaner)

<h2 id="ImageCleaner" class="doc_header"><code>class</code> <code>ImageCleaner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L97" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>ImageCleaner</code>(**`dataset`**, **`fns_idxs`**, **`path`**, **`batch_size`**:`int`=***`5`***, **`duplicates`**=***`False`***)

<div class="collapse" id="ImageCleaner-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>ImageCleaner</code>:</p><ul><li><code>pytest -sv tests/test_widgets_image_cleaner.py::test_image_cleaner_index_length_mismatch</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_widgets_image_cleaner.py#L16" class="source_link" style="float:right">[source]</a></li><li><code>pytest -sv tests/test_widgets_image_cleaner.py::test_image_cleaner_length_correct</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_widgets_image_cleaner.py#L23" class="source_link" style="float:right">[source]</a></li><li><code>pytest -sv tests/test_widgets_image_cleaner.py::test_image_cleaner_wrong_input_type</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_widgets_image_cleaner.py#L30" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Displays images for relabeling or deletion and saves changes in `path` as 'cleaned.csv'.  

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) is for cleaning up images that don't belong in your dataset. It renders images in a row and gives you the opportunity to delete the file from your file system. To use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) we must first use `DatasetFormatter().from_toplosses` to get the suggested indices for misclassified images.

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn)

In [ ]:
ImageCleaner(ds, idxs, path)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) does not change anything on disk (neither labels or existence of images). Instead, it creates a 'cleaned.csv' file in your data path from which you need to load your new databunch for the files to changes to be applied. 

In [ ]:
df = pd.read_csv(path/'cleaned.csv', header='infer')

In [ ]:
# We create a databunch from our csv. We include the data in the training set and we don't use a validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
db = (ImageList.from_df(df, path)
                   .split_none()
                   .label_from_df()
                   .databunch(bs=64))

In [ ]:
learn = cnn_learner(db, models.resnet18, metrics=error_rate)
learn = learn.load('stage-1')

You can then use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) again to find duplicates in the dataset. To do this, you can specify `duplicates=True` while calling ImageCleaner after getting the indices and dataset from `.from_similars`. Note that if you are using a layer's output which has dimensions <code>(n_batches, n_features, 1, 1)</code> then you don't need any pooling (this is the case with the last layer). The suggested use of `.from_similars()` with resnets is using the last layer and no pooling, like in the following cell.

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn, layer_ls=[0,7,1], pool=None)

Getting activations...


Computing similarities...


In [ ]:
ImageCleaner(ds, idxs, path, duplicates=True)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

In [ ]:
show_doc(ImageDownloader)

<h2 id="ImageDownloader" class="doc_header"><code>class</code> <code>ImageDownloader</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_downloader.py#L16" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageDownloader-pytest" style="float:right; padding-right:10px">[test]</a></h2>

> <code>ImageDownloader</code>(**`path`**:`PathOrStr`=***`'data'`***)

<div class="collapse" id="ImageDownloader-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageDownloader-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>Tests found for <code>ImageDownloader</code>:</p><ul><li><code>pytest -sv tests/test_widgets_image_cleaner.py::test_image_downloader_with_path</code> <a href="https://github.com/fastai/fastai/blob/master/tests/test_widgets_image_cleaner.py#L36" class="source_link" style="float:right">[source]</a></li></ul><p>To run tests please refer to this <a href="/dev/test.html#quick-guide">guide</a>.</p></div></div>

Displays a widget that allows searching and downloading images from google images search in a Jupyter Notebook or Lab. 

[`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) widget gives you a way to quickly bootstrap your image dataset without leaving the notebook. It searches and downloads images that match the search criteria and resolution / quality requirements and stores them on your filesystem within the provided `path`.

Images for each search query (or label) are stored in a separate folder within `path`. For example, if you pupulate `tiger` with a `path` setup to `./data`, you'll get a folder `./data/tiger/` with the tiger images in it.

[`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) will automatically clean up and verify the downloaded images with [`verify_images()`](/vision.data.html#verify_images) after downloading them.

In [ ]:
path = Config.data_path()/'image_downloader'
os.makedirs(path, exist_ok=True)
ImageDownloader(path)

#### Downloading images in python scripts outside Jupyter notebooks

In [ ]:
path = Config.data_path()/'image_downloader'
files = download_google_images(path, 'aussie shepherd', size='>1024*768', n_images=30)

len(files)

30

In [ ]:
show_doc(download_google_images)

<h4 id="download_google_images" class="doc_header"><code>download_google_images</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_downloader.py#L78" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#download_google_images-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>download_google_images</code>(**`path`**:`PathOrStr`, **`search_term`**:`str`, **`size`**:`str`=***`'>400*300'`***, **`n_images`**:`int`=***`10`***, **`format`**:`str`=***`'jpg'`***, **`max_workers`**:`int`=***`16`***, **`timeout`**:`int`=***`4`***) → `FilePathList`

<div class="collapse" id="download_google_images-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#download_google_images-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>download_google_images</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Search for `n_images` images on Google, matching `search_term` and `size` requirements, download them into `path`/`search_term` and verify them, using `max_workers` threads. 

After populating images with [`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader), you can get a an [`ImageDataBunch`](/vision.data.html#ImageDataBunch) by calling `ImageDataBunch.from_folder(path, size=size)`, or using the data block API.

In [ ]:
# Setup path and labels to search for
path = Config.data_path()/'image_downloader'
labels = ['boston terrier', 'french bulldog']

# Download images
for label in labels: 
    download_google_images(path, label, size='>400*300', n_images=50)

# Build a databunch and train! 
src = (ImageList.from_folder(path)
       .split_by_rand_pct()
       .label_from_folder()
       .transform(get_transforms(), size=224))

db  = src.databunch(bs=16, num_workers=0)

learn = cnn_learner(db, models.resnet34, metrics=[accuracy])
learn.fit_one_cycle(3)

cannot identify image file <_io.BufferedReader name='/home/ubuntu/.fastai/data/image_downloader/boston terrier/00000044.jpg'>
cannot identify image file <_io.BufferedReader name='/home/ubuntu/.fastai/data/image_downloader/boston terrier/00000014.jpg'>


ConnectionError: None: Max retries exceeded with url: /wp-content/uploads/2018/04/cropped-frenchbulldog2-french840-2.jpg (Caused by None)

#### Downloading more than a hundred images

To fetch more than a hundred images, [`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) uses `selenium` and `chromedriver` to scroll through the Google Images search results page and scrape image URLs. They're not required as dependencies by default. If you don't have them installed on your system, the widget will show you an error message.

To install `selenium`, just `pip install selenium` in your fastai environment.

**On a mac**, you can install `chromedriver` with `brew cask install chromedriver`.

**On Ubuntu**
Take a look at the latest Chromedriver version available, then something like:

```
wget https://chromedriver.storage.googleapis.com/2.45/chromedriver_linux64.zip
unzip chromedriver_linux64.zip
```

Note that downloading under 100 images doesn't require any dependencies other than fastai itself, however downloading more than a hundred images [uses `selenium` and `chromedriver`](/widgets.image_cleaner.html#Downloading-more-than-a-hundred-images).

`size` can be one of:

```
'>400*300'
'>640*480'
'>800*600'
'>1024*768'
'>2MP'
'>4MP'
'>6MP'
'>8MP'
'>10MP'
'>12MP'
'>15MP'
'>20MP'
'>40MP'
'>70MP'
```

## Methods

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(ImageCleaner.make_dropdown_widget)

<h4 id="ImageCleaner.make_dropdown_widget" class="doc_header"><code>make_dropdown_widget</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L127" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-make_dropdown_widget-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>make_dropdown_widget</code>(**`description`**=***`'Description'`***, **`options`**=***`['Label 1', 'Label 2']`***, **`value`**=***`'Label 1'`***, **`file_path`**=***`None`***, **`layout`**=***`Layout()`***, **`handler`**=***`None`***)

<div class="collapse" id="ImageCleaner-make_dropdown_widget-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-make_dropdown_widget-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>make_dropdown_widget</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return a Dropdown widget with specified `handler`.  

In [ ]:
show_doc(ImageCleaner.next_batch)

<h4 id="ImageCleaner.next_batch" class="doc_header"><code>next_batch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L165" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-next_batch-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>next_batch</code>(**`_`**)

<div class="collapse" id="ImageCleaner-next_batch-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-next_batch-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>next_batch</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Handler for 'Next Batch' button click. Delete all flagged images and renders next batch.  

In [ ]:
show_doc(DatasetFormatter.sort_idxs)

<h4 id="DatasetFormatter.sort_idxs" class="doc_header"><code>sort_idxs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L90" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-sort_idxs-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>sort_idxs</code>(**`similarities`**)

<div class="collapse" id="DatasetFormatter-sort_idxs-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-sort_idxs-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>sort_idxs</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Sorts `similarities` and return the indexes in pairs ordered by highest similarity.  

In [ ]:
show_doc(ImageCleaner.make_vertical_box)

<h4 id="ImageCleaner.make_vertical_box" class="doc_header"><code>make_vertical_box</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L141" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-make_vertical_box-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>make_vertical_box</code>(**`children`**, **`layout`**=***`Layout()`***, **`duplicates`**=***`False`***)

<div class="collapse" id="ImageCleaner-make_vertical_box-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-make_vertical_box-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>make_vertical_box</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Make a vertical box with [`children`](/torch_core.html#children) and `layout`.  

In [ ]:
show_doc(ImageCleaner.relabel)

<h4 id="ImageCleaner.relabel" class="doc_header"><code>relabel</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L158" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-relabel-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>relabel</code>(**`change`**)

<div class="collapse" id="ImageCleaner-relabel-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-relabel-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>relabel</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Relabel images by moving from parent dir with old label `class_old` to parent dir with new label `class_new`.  

In [ ]:
show_doc(DatasetFormatter.largest_indices)

<h4 id="DatasetFormatter.largest_indices" class="doc_header"><code>largest_indices</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L82" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-largest_indices-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>largest_indices</code>(**`arr`**, **`n`**)

<div class="collapse" id="DatasetFormatter-largest_indices-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-largest_indices-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>largest_indices</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Returns the `n` largest indices from a numpy array `arr`.  

In [ ]:
show_doc(ImageCleaner.delete_image)

<h4 id="ImageCleaner.delete_image" class="doc_header"><code>delete_image</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L183" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-delete_image-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>delete_image</code>(**`file_path`**)

<div class="collapse" id="ImageCleaner-delete_image-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-delete_image-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>delete_image</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(ImageCleaner.empty)

<h4 id="ImageCleaner.empty" class="doc_header"><code>empty</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L186" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-empty-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>empty</code>()

<div class="collapse" id="ImageCleaner-empty-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-empty-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>empty</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(ImageCleaner.empty_batch)

<h4 id="ImageCleaner.empty_batch" class="doc_header"><code>empty_batch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L181" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-empty_batch-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>empty_batch</code>()

<div class="collapse" id="ImageCleaner-empty_batch-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-empty_batch-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>empty_batch</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(DatasetFormatter.comb_similarity)

<h4 id="DatasetFormatter.comb_similarity" class="doc_header"><code>comb_similarity</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L70" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-comb_similarity-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>comb_similarity</code>(**`t1`**:`Tensor`, **`t2`**:`Tensor`, **\*\*`kwargs`**)

<div class="collapse" id="DatasetFormatter-comb_similarity-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-comb_similarity-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>comb_similarity</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Computes the similarity function between each embedding of `t1` and `t2` matrices.  

In [ ]:
show_doc(ImageCleaner.get_widgets)

<h4 id="ImageCleaner.get_widgets" class="doc_header"><code>get_widgets</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L189" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-get_widgets-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get_widgets</code>(**`duplicates`**)

<div class="collapse" id="ImageCleaner-get_widgets-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-get_widgets-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get_widgets</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create and format widget set.  

In [ ]:
show_doc(ImageCleaner.write_csv)

<h4 id="ImageCleaner.write_csv" class="doc_header"><code>write_csv</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L209" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-write_csv-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>write_csv</code>()

<div class="collapse" id="ImageCleaner-write_csv-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-write_csv-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>write_csv</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

In [ ]:
show_doc(ImageCleaner.create_image_list)

<h4 id="ImageCleaner.create_image_list" class="doc_header"><code>create_image_list</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L147" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-create_image_list-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>create_image_list</code>(**`dataset`**, **`fns_idxs`**)

<div class="collapse" id="ImageCleaner-create_image_list-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-create_image_list-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>create_image_list</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Create a list of images, filenames and labels but first removing files that are not supposed to be displayed.  

In [ ]:
show_doc(ImageCleaner.render)

<h4 id="ImageCleaner.render" class="doc_header"><code>render</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L220" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-render-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>render</code>()

<div class="collapse" id="ImageCleaner-render-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-render-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>render</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Re-render Jupyter cell for batch of images.  

In [ ]:
show_doc(DatasetFormatter.get_similars_idxs)

<h4 id="DatasetFormatter.get_similars_idxs" class="doc_header"><code>get_similars_idxs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L41" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-get_similars_idxs-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get_similars_idxs</code>(**`learn`**, **`layer_ls`**, **\*\*`kwargs`**)

<div class="collapse" id="DatasetFormatter-get_similars_idxs-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-get_similars_idxs-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get_similars_idxs</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Gets the indices for the most similar images in `ds_type` dataset  

In [ ]:
show_doc(ImageCleaner.on_delete)

<h4 id="ImageCleaner.on_delete" class="doc_header"><code>on_delete</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L176" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-on_delete-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>on_delete</code>(**`btn`**)

<div class="collapse" id="ImageCleaner-on_delete-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-on_delete-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>on_delete</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Flag this image as delete or keep.  

In [ ]:
show_doc(ImageCleaner.make_button_widget)

<h4 id="ImageCleaner.make_button_widget" class="doc_header"><code>make_button_widget</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L117" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-make_button_widget-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>make_button_widget</code>(**`label`**, **`file_path`**=***`None`***, **`handler`**=***`None`***, **`style`**=***`None`***, **`layout`**=***`Layout(width='auto')`***)

<div class="collapse" id="ImageCleaner-make_button_widget-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-make_button_widget-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>make_button_widget</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Return a Button widget with specified `handler`.  

In [ ]:
show_doc(ImageCleaner.make_img_widget)

<h4 id="ImageCleaner.make_img_widget" class="doc_header"><code>make_img_widget</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L112" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-make_img_widget-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>make_img_widget</code>(**`img`**, **`layout`**=***`Layout()`***, **`format`**=***`'jpg'`***)

<div class="collapse" id="ImageCleaner-make_img_widget-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-make_img_widget-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>make_img_widget</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Returns an image widget for specified file name `img`.  

In [ ]:
show_doc(DatasetFormatter.get_actns)

<h4 id="DatasetFormatter.get_actns" class="doc_header"><code>get_actns</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L52" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-get_actns-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get_actns</code>(**`learn`**, **`hook`**:[`Hook`](/callbacks.hooks.html#Hook), **`dl`**:[`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), **`pool`**=***`'AdaptiveConcatPool2d'`***, **`pool_dim`**:`int`=***`4`***, **\*\*`kwargs`**)

<div class="collapse" id="DatasetFormatter-get_actns-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-get_actns-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get_actns</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Gets activations at the layer specified by `hook`, applies `pool` of dim `pool_dim` and concatenates  

In [ ]:
show_doc(ImageCleaner.batch_contains_deleted)

<h4 id="ImageCleaner.batch_contains_deleted" class="doc_header"><code>batch_contains_deleted</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L203" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-batch_contains_deleted-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>batch_contains_deleted</code>()

<div class="collapse" id="ImageCleaner-batch_contains_deleted-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-batch_contains_deleted-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>batch_contains_deleted</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Check if current batch contains already deleted images.  

In [ ]:
show_doc(ImageCleaner.make_horizontal_box)

<h4 id="ImageCleaner.make_horizontal_box" class="doc_header"><code>make_horizontal_box</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L136" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#ImageCleaner-make_horizontal_box-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>make_horizontal_box</code>(**`children`**, **`layout`**=***`Layout()`***)

<div class="collapse" id="ImageCleaner-make_horizontal_box-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#ImageCleaner-make_horizontal_box-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>make_horizontal_box</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Make a horizontal box with [`children`](/torch_core.html#children) and `layout`.  

In [ ]:
show_doc(DatasetFormatter.get_toplosses_idxs)

<h4 id="DatasetFormatter.get_toplosses_idxs" class="doc_header"><code>get_toplosses_idxs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L22" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-get_toplosses_idxs-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>get_toplosses_idxs</code>(**`learn`**, **`n_imgs`**, **\*\*`kwargs`**)

<div class="collapse" id="DatasetFormatter-get_toplosses_idxs-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-get_toplosses_idxs-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>get_toplosses_idxs</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

Sorts `ds_type` dataset by top losses and returns dataset and sorted indices.  

In [ ]:
show_doc(DatasetFormatter.padded_ds)

<h4 id="DatasetFormatter.padded_ds" class="doc_header"><code>padded_ds</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L31" class="source_link" style="float:right">[source]</a><a class="source_link" data-toggle="collapse" data-target="#DatasetFormatter-padded_ds-pytest" style="float:right; padding-right:10px">[test]</a></h4>

> <code>padded_ds</code>(**`ll_input`**, **`size`**=***`(250, 300)`***, **`resize_method`**=***`<ResizeMethod.CROP: 1>`***, **`padding_mode`**=***`'zeros'`***, **\*\*`kwargs`**)

<div class="collapse" id="DatasetFormatter-padded_ds-pytest"><div class="card card-body pytest_card"><a type="button" data-toggle="collapse" data-target="#DatasetFormatter-padded_ds-pytest" class="close" aria-label="Close"><span aria-hidden="true">&times;</span></a><p>No tests found for <code>padded_ds</code>. To contribute a test please refer to <a href="/dev/test.html">this guide</a> and <a href="https://forums.fast.ai/t/improving-expanding-functional-tests/32929">this discussion</a>.</p></div></div>

For a LabelList `ll_input`, resize each image to `size` using `resize_method` and `padding_mode`.  

## New Methods - Please document or move to the undocumented section